13/06/2021: Actualice el codigo, agregue unos diccionarios y cambie el nombre de las variables. Pero no lo he probado

In [1]:
import pandas as pd

#Dado el camino de un archivo, devuelvo su contenido como un DataFrame de Pandas, donde cada texto es un tweet sin procesar
def abrirArchivo(filepath):
    df = pd.read_csv(filepath, sep='\t', encoding='utf-8').dropna()
    
    return df

In [2]:
#Dado un DataFrame y score minimo
#Devuelvo un dataframe que contiene solo las filas con scores mayores al minimo
def remove_low_score_rows(df_tweets, min_score):
    df_tweets = df_tweets[df_tweets._score>min_score]
    df_tweets = df_tweets.drop(columns=['_score'])
    return df_tweets

In [3]:
#Defino funciones para eliminar los elementos innecesarios de los textos
import regex as re

#Dado un DataFrame con una lista de frases, filtro sus hashtags
def remove_hashtags(df_sentences):
    for i in range(len(df_sentences)):
        df_sentences.loc[i, ('text')] = re.sub(r'#(\s)?(\w)+|(#)', '', df_sentences.loc[i, ('text')])

#Dado un DataFrame con una lista de frases, filtro 
 #el RT @Usuario al inicio del campo texto de los retweets y los @Usuario en cualquier parte del texto
def remove_rt(df_sentences):
    for i in range(len(df_sentences)):
        df_sentences.loc[i, ('text')] = re.sub(r'(rt @\w+(:\s)?)|(@\w+)|(@)|(RT @\w+(:\s)?)', '', df_sentences.loc[i, ('text')])

#Dado un DataFrame con una lista de frases, filtro las URLs
def remove_url(df_sentences):
    for i in range(len(df_sentences)):
        df_sentences.loc[i, ('text')] = re.sub(r'(http(s)?://.*)+', '', df_sentences.loc[i, ('text')])

#Dado un DataFrame con una lista de frases, filtro los signos de puntuacion
def remove_punctuation(df_sentences):
    for i in range(len(df_sentences)):
        df_sentences.loc[i, ('text')] = re.sub(r'[\\\.,;:!¡?¿\-_\“\”\"\‘\’\'\s$\)\(\%]|(\'s)', ' ', df_sentences.loc[i, ('text')])
        df_sentences.loc[i, ('text')] = re.sub(r'(\s*\n\s*)+|(\s)+', ' ', df_sentences.loc[i, ('text')])

#Dado un DataFrame con una lista de frases, llamo a todos los filtros definidos anteriormente 
def remove_all(df_sentences):
    remove_rt(df_sentences)
    remove_hashtags(df_sentences)
    remove_url(df_sentences)
    remove_punctuation(df_sentences)

In [4]:
#Quito las stopwords
import spacy
#from nltk.corpus import stopwords

#Dado un DataFrame con una lista de frases, filtro las stopwords
def remove_stopwords(df_sentences, spacy_model):
    #stop_words = set(stopwords.words(lang))
    stop_words = spacy.load(spacy_model).Defaults.stop_words
    stop_words.add('amp')
    filtered_words = []
    
    for i in range(len(df_sentences)):
        words_wo_stop_words = list()
        words = df_sentences.loc[i, ('text')].split()
        for word in words:
            if(word.lower() not in stop_words) and (len(word) > 1):
                words_wo_stop_words.append(word)
        df_sentences.loc[i, ('text')] = ' '.join(words_wo_stop_words)

In [5]:
import spacy
import lemminflect

#Dado un DataFrame con una lista de frases y un modelo de spacy, paso palabras a sus lemmas
def transform_lemmas(df_sentences, spacy_model):
    nlp = spacy.load(spacy_model)

    for i in range(len(df_sentences)):
        words_lemmas = list()
        words_nlp = nlp(df_sentences.loc[i, ('text')])
        for word in words_nlp:
            words_lemmas.append(word._.lemma())
        df_sentences.loc[i, ('text')] = ' '.join(words_lemmas)

In [6]:
#Dado un DataFrame con una lista de frases, remuevo las frases vacias
def remove_empty(df_tweets):
    df_tweets = df_tweets.replace(r'( )+',' ', regex=True)
    df_tweets = df_tweets.replace(r'(\n)+','\n', regex=True)
    df_tweets = df_tweets.dropna(how='all')
    
    return df_tweets

In [7]:
#Dado un DataFrame con una lista de frases, remuevo las frases duplicadas
def remove_duplicates(df_tweets):
    return df_tweets.drop_duplicates(subset=['text'])

In [8]:
#Recibo una lista de frases y datos sobre su procedencia y las escribo en un archivo
def escribirArchivoTweetsProc(df_tweets, name_file, year, lang):
    path_proc = 'results_processed/'
    filepath = path_proc + str(year) + '/' + str(lang)+ '/' + name_file
    df_tweets.to_csv(filepath, encoding='utf-8', index=False, sep='\t')

In [9]:
#TEMP
def escribirArchivoToLabelProc(df_tweets, name_file, lang):
    path_proc = 'toLabel//proc'
    filepath = path_proc + '//' + str(lang)+ '//' + name_file
    df_tweets.to_csv(filepath, encoding='utf-8', index=False, sep='\t')

Automatizo el proceso de abrir los archivos y procesarlos

In [10]:
cant_temas = 4
years = [2018,2020,2021]
years = [2021]
langs = ['pt','es','en']
min_score = 0
path_results = 'results/'
dic_languages = {'en': 'english', 'es': 'spanish', 'pt': 'portuguese'}
dic_spacy_models = {'en': 'en_core_web_lg', 'es': 'es_core_news_lg', 'pt': 'pt_core_news_lg'}

#Separo los paises segun el lenguaje de sus tweets
dic_countries = {'en': ['alemania', 'arabia', 'australia', 'brasil', 'canada', 'china', 'corea del sur', 'francia', 
                        'india', 'indonesia', 'italia', 'japon', 'uk', 'rusia', 'sudafrica', 'turquia', 'eu', 'usa'],
                'es': ['argentina', 'mexico'],
                'pt':['brasil']}

In [11]:
%load_ext jupyternotify

for year_g20 in years:
    for lang in langs:
        list_countries = dic_countries[lang]
        lang_stopwords = dic_languages[lang]
        spacy_model = dic_spacy_models[lang]
        for country in list_countries:
            for subject in range(cant_temas):
                print('year: '+str(year_g20)+' lang: '+str(lang)+' country: '+str(country)+' subject: '+str(subject))
                name_file = country + ' ' + str(year_g20) + ' - Tema ' + str(subject) + '.csv'
                filepath = path_results + str(year_g20) + '/' + lang + '/' + name_file
                df_tweets = abrirArchivo(filepath)
                df_tweets = remove_low_score_rows(df_tweets, min_score)
                remove_all(df_tweets)
                remove_stopwords(df_tweets, spacy_model)
                transform_lemmas(df_tweets, spacy_model)
                df_tweets = remove_empty(df_tweets)
                df_tweets = remove_duplicates(df_tweets)
                escribirArchivoTweetsProc(df_tweets, name_file, year_g20, lang)
                
%notify

<IPython.core.display.Javascript object>

year: 2021 lang: pt country: brasil subject: 0
year: 2021 lang: pt country: brasil subject: 1
year: 2021 lang: pt country: brasil subject: 2
year: 2021 lang: pt country: brasil subject: 3
year: 2021 lang: es country: argentina subject: 0
year: 2021 lang: es country: argentina subject: 1
year: 2021 lang: es country: argentina subject: 2
year: 2021 lang: es country: argentina subject: 3
year: 2021 lang: es country: mexico subject: 0
year: 2021 lang: es country: mexico subject: 1
year: 2021 lang: es country: mexico subject: 2
year: 2021 lang: es country: mexico subject: 3
year: 2021 lang: en country: alemania subject: 0
year: 2021 lang: en country: alemania subject: 1
year: 2021 lang: en country: alemania subject: 2
year: 2021 lang: en country: alemania subject: 3
year: 2021 lang: en country: arabia subject: 0
year: 2021 lang: en country: arabia subject: 1
year: 2021 lang: en country: arabia subject: 2
year: 2021 lang: en country: arabia subject: 3
year: 2021 lang: en country: australia s

<IPython.core.display.Javascript object>

In [22]:
#TEMP
%load_ext jupyternotify

path = 'toLabel\\'
names = ['Ana', 'Axel', 'Fernando', 'Mauro']
langs = ['es','en']

for name in names:
    for lang in langs:
        lang_stopwords = dic_languages[lang]
        spacy_model = dic_spacy_models[lang]
        name_file = 'toLabel-other-' + name + '-' + lang +'.csv'
        filepath = path + lang + '\\' + name_file
        df_tweets = abrirArchivo(filepath)
        #df_tweets = remove_low_score_rows(df_tweets, min_score)
        remove_all(df_tweets)
        remove_stopwords(df_tweets, spacy_model)
        transform_lemmas(df_tweets, spacy_model)
        df_tweets = remove_empty(df_tweets)
        df_tweets = remove_duplicates(df_tweets)
        escribirArchivoToLabelProc(df_tweets, name_file, lang)
                
%notify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


<IPython.core.display.Javascript object>

In [25]:
#TEMP
%load_ext jupyternotify

path = 'toLabel\\'
langs = ['es','en']

for lang in langs:
    lang_stopwords = dic_languages[lang]
    spacy_model = dic_spacy_models[lang]
    name_file = 'toLabel-common-' + lang +'.csv'
    filepath = path + lang + '\\' + name_file
    df_tweets = abrirArchivo(filepath)
    remove_all(df_tweets)
    remove_stopwords(df_tweets, spacy_model)
    transform_lemmas(df_tweets, spacy_model)
    df_tweets = remove_empty(df_tweets)
    df_tweets = remove_duplicates(df_tweets)
    escribirArchivoToLabelProc(df_tweets, name_file, lang)
    
%notify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


<IPython.core.display.Javascript object>